In [1]:
# import dependencies
import pandas as pd
import datetime
from dateutil import parser
import numpy as np


In [2]:
# read csv with data for open mic events
OMs = pd.read_csv('TCB_mics_calendar - TCB_mics_calendar copy 6.csv')

# remove rows that don't have a name or a venue
OMs = OMs.dropna(subset=['EVENT NAME', 'VENUE'])

# dictionary with adjusted column names
columns = {}

# list with columns to be dropped
drops = []

# adjust column names and determine which columns should be dropped
for col in OMs.columns:
    c = col.replace(' - MIC', '').replace('CATEGORIES', 'CATEGORY').replace('?', '').replace(' MICS', '').replace(' ', '_').lower()
    if '(dont_fill)' in c:
        drops.append(c)
    columns[col] = c

# set column names to adjusted column names
OMs = OMs.rename(columns=columns)

# remove columns that aren't supposed to be filled
OMs = OMs.drop(columns=drops)

# display dataframe
OMs


,event_name,venue,neighborhood,start_date,start_time,end_date,end_time,all_day_event,category,event_cost,purchase_req,free,show_map_link,show_map,event_description,stage_time,event_website
0,(11AM-12PM Hourly) Improv Mic @ Fourth Wall Co...,Fourth Wall Comedy Cafe,Hollywood/East Hollywood,2020-03-15,11:00 AM,2020-03-15,1:45 PM,False,Open Mic,$5,No min.,NaN,True,True,For stand-up riffing and crowd work\nSuggestio...,5 min.,https://slotted.co/fwcafe
1,Improv Mic,The Hollywood Comedy,Hollywood/East Hollywood,2020-03-15,11:00 AM,2020-03-15,12:00 PM,False,Improvised stand-up mic,$5,No min.,NaN,True,True,includes snacks and drinks\nSign-up in advance...,5 min.,https://slotted.co/thehollywoodcomedy
2,(1PM-6PM Hourly) The Hollywood Comedy,The Hollywood Comedy,Hollywood/East Hollywood,2020-03-08,1:00 PM,2020-03-08,7:00 PM,False,Open Mic,$5,No min.,NaN,True,True,includes snacks & drinks\nSign-up in advance a...,5 min.,https://slotted.co/thehollywoodcomedy
3,(12PM-1PM Hourly) Flashback Mic (in Melrose),Flashback Mic (on Melrose),WeHo/Fairfax/Beverly Hills,2020-03-15,12:00 PM,2020-03-15,2:00 PM,False,Open Mic,$5,No min.,NaN,True,True,"*entrance located in rear, look for the red do...",5 min.,https://slotted.co/fbmic
4,(1PM-12AM Hourly) 15 Mins. The Hollywood Comedy,The Hollywood Comedy,Hollywood/East Hollywood,2020-03-08,1:00 PM,2020-03-08,11:59 PM,False,Open Mic,$12,No min.,NaN,True,True,"\n\nin ""The Fancy Room"" (up front)\n$12 (inclu...",15 min.,https://slotted.co/thehollywoodcomedy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,Geeky Teas & Games,Geeky Teas & Games,Burbank/NoHo/Valley,2020-02-22,9:30 PM,2020-02-22,11:30 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes bottled water\nSign-up 9PM\nStart...,5 min.,NaN
457,Santa Monica Playhouse,Santa Monica Playhouse,Westside/Santa Monica,2020-02-29,9:30 PM,2020-02-29,12:00 AM,False,Open Mic,$5,No min.,NaN,True,True,"\n\nNot on Main Stage, in Other Theatre\nPassw...",5 min.,NaN
458,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-14,10:00 PM,2020-03-14,11:00 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
459,Atomic Wombat Comics,Atomic Wombat Comics,Orange County,2020-03-07,10:30 PM,2020-03-07,11:30 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nSign-up in advance at https://slotted.co/c...,5 min.,https://slotted.co/comedygrabbag


In [3]:
# list to hold time related columns
begin_and_end = ['start_date', 'end_date', 'start_time', 'end_time']

# convert data in these columns from strings to timestamps
for i , t in enumerate(begin_and_end):
    if i < 2:
        OMs[t] = OMs[t].apply(lambda x: parser.parse(x))
    else:
        OMs[t] = OMs[t].apply(lambda x: parser.parse(x).hour)
        
# sort events in chronological order
OMs = OMs.sort_values(['start_date', 'start_time'])


In [4]:

def time_string(hour):
    if hour > 12:
        if hour != 24:
            hour = str(hour-12)+'PM'
        else:
            hour = '12AM'
    else:
        if hour != 12:
            hour = str(hour)+'AM'
        else: hour = '12PM'
    return hour


In [5]:
# # list to hold names of all unique events
# events = OMs['event_name'].unique()


    
# # all events that share a particular name
# event = OMs.copy().loc[OMs['event_name'] == 'Echoes on Pico',:]

# # list to hold the indicies of events of a shared name
# rows = list(event.index)

# # consolidate times
# new = 0
# start = event['start_time'][rows[0]]
# for i in range(1, len(rows)):
#     end = event[f'end_time'][rows[i]]
#     if event['start_time'][rows[i]] == event['end_time'][rows[i-1]]:
#         OMs.loc[rows[i-1: i+1], 'new_start_time'] = start
#         OMs.loc[rows[new:i+1], 'new_end_time'] = end
#     else:
#         new = i
#         start = event['start_time'][rows[i]]

# # consolidate dates
# new = 0
# start = event['start_date'][rows[0]]
# for i in range(1, len(rows)):
#     end = event[f'end_date'][rows[i]]
#     if (event['start_date'][rows[i]] - event['end_date'][rows[i-1]]).days < 2:
#         OMs.loc[rows[i-1: i+1], 'new_start_date'] = start
#         OMs.loc[rows[new:i+1], 'new_end_date'] = end
#     else:
#         new = i
#         start = event['start_date'][rows[i]]
        
        

# pico2 = OMs.copy().loc[OMs['event_name'] == 'Echoes on Pico',:].reset_index(drop=True)

# # for d in pico['event_description']:
# #     print(d)

# pico2



In [6]:
# list to hold names of all unique events
events = OMs['event_name'].unique()

for e in events:
    
    # all events that share a particular name
    event = OMs.copy().loc[OMs['event_name'] == e,:]
    
    # list to hold the indicies of events of a shared name
    rows = list(event.index)

    # consolidate times
    new = 0
    start = event['start_time'][rows[0]]
    for i in range(1, len(rows)):
        end = event[f'end_time'][rows[i]]
        if event['start_time'][rows[i]] == event['end_time'][rows[i-1]]:
            OMs.loc[rows[i-1: i+1], 'new_start_time'] = start
            OMs.loc[rows[new:i+1], 'new_end_time'] = end
        else:
            new = i
            start = event['start_time'][rows[i]]

    # consolidate dates
    new = 0
    start = event['start_date'][rows[0]]
    for i in range(1, len(rows)):
        end = event[f'end_date'][rows[i]]
        if (event['start_date'][rows[i]] - event['end_date'][rows[i-1]]).days < 2:
            OMs.loc[rows[i-1: i+1], 'new_start_date'] = start
            OMs.loc[rows[new:i+1], 'new_end_date'] = end
        else:
            new = i
            start = event['start_date'][rows[i]]


In [7]:
OMs[['new_start_time', 'new_end_time']] = OMs[['new_start_time', 'new_end_time']].astype('int32')

OMs[['new_start_time', 'new_end_time']]

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:


OMs['event_description'] = OMs.apply(lambda x: x['event_description'].replace(time_string(x['start_time']), time_string(x['new_start_time'])), axis=1)
OMs['event_description'] = OMs.apply(lambda x: x['event_description'].replace(time_string(x['end_time']), time_string(x['new_end_time'])), axis=1)




In [ ]:
for bne in begin_and_end:
    
    OMs = OMs.drop(columns=bne)
    OMs = OMs.rename(columns={'new_' + bne: bne})

# drop rows with the same information
OMs = OMs.drop_duplicates().reset_index(drop=True)


In [ ]:
# display final dataframe
OMs
# OMs['event_description'][0]

In [ ]:
# for e in OMs['event_name'].value_counts():
#     print(e)

In [ ]:
OMs.copy().loc[OMs['event_name'] == 'Echoes on Pico',:].reset_index(drop=True)['event_description'][1]

In [ ]:
len(OMs)